In [1]:
%%bash

pip install -U langchain sentence_transformers chromadb openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.


In [2]:
import json
with open("sample_data/extracted.json", "r") as file:
    data = json.load(file)

In [6]:
from getpass import getpass
OPENAI_API_KEY = getpass("Enter your OpenAI key: ")

Enter your OpenAI key: ··········


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
import chromadb
from langchain.vectorstores import Chroma

texts = []
metadatas = []
for key, value in data.items():
  metadatas.append({"heading": key})
  texts.append(value)
retriever = Chroma.from_texts(texts=texts, embedding=embeddings, metadatas=metadatas).as_retriever(
    search_kwargs={"k": 3}
)

In [7]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

In [8]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_llm(llm=llm, retriever=retriever)

In [9]:
chain.run({"query": "Can a person complete requried course credits by just doing lab courses?"})

' No, a person cannot complete the required course credits by just doing lab courses. The curriculum is designed to allow students to take additional courses, but the minimum credit requirements must be met by taking theory courses, laboratory courses, and/or other academic activities like seminars and projects.'